In [9]:
import sys
import os
import pandas as pd
import numpy as np
from itertools import product

notebook_dir = os.path.dirname(os.path.realpath("__file__"))
project_root_dir = os.path.join(notebook_dir, '..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

from logic.mortgage_repayments import compute_monthly_mortgage_repayments
from logic.income_calculations import compute_income_after_essentials, compute_income_after_total_taxation, compute_income_after_paye
from logic.paye_taxation import compute_student_loan_repayments

In [11]:
annual_pension_contributions_rate = 0.05

annual_council_tax = 1_475
annual_home_insrance = 200
annual_ground_rent = 50
annual_service_charge = 2500

annual_minimum_additional_living_costs_per_person = 8_500
annual_inflated_additional_minimum_additional_living_costs_per_person = 13_000

mortgage_length_in_years = 25

In [8]:
def compute_monthly_income_left_for_mortgage_per_month(
    salary,
    pension_contributions_rate,
    annual_council_tax,
    annual_home_insurance,
    annual_ground_rent,
    annual_service_charge,
    lifestyle,
    annual_additional_minimum_living_costs,
    annual_inflated_additional_minimum_living_costs
):
    income_after_total_taxation = compute_income_after_total_taxation(salary, pension_contributions_rate, annual_council_tax)
    annual_additional_housing_costs = annual_home_insurance + annual_ground_rent + annual_service_charge

    if lifestyle == "45p_net_on_mortgage":
        income_left_for_mortgage = 0.45 * income_after_total_taxation
    elif lifestyle == "35p_net_on_mortgage":
        income_left_for_mortgage = 0.35 * income_after_total_taxation
    elif lifestyle == "28p_net_on_housing":
        income_left_for_housing = 0.28 * income_after_total_taxation
        income_left_for_mortgage = income_left_for_housing - annual_home_insurance - annual_ground_rent - annual_service_charge
    elif lifestyle == "25p_net_on_mortgage":
        income_left_for_mortgage = 0.25 * income_after_total_taxation
    elif lifestyle == "25p_gross_on_debts":
        income_left_for_mortgage = 0.25 * salary - compute_student_loan_repayments(salary)
    elif lifestyle == "50-30-20":
        income_left_for_mortgage = income_after_total_taxation*0.5 - annual_additional_housing_costs - annual_additional_minimum_living_costs
    elif lifestyle == "inflated_75-25":
        income_left_for_mortgage = income_after_total_taxation*0.75 - annual_additional_housing_costs - annual_inflated_additional_minimum_living_costs
    elif lifestyle == "current_essentials":
        income_left_for_mortgage = compute_income_after_essentials(
            salary,
            pension_contributions_rate,
            annual_council_tax,
            annual_home_insurance,
            annual_ground_rent,
            annual_service_charge,
            annual_additional_minimum_living_costs
        )
    elif lifestyle == "inflated_essentials":
        income_left_for_mortgage = compute_income_after_essentials(
            salary,
            pension_contributions_rate,
            annual_council_tax,
            annual_home_insurance,
            annual_ground_rent,
            annual_service_charge,
            annual_inflated_additional_minimum_living_costs
        )
    else:
        raise Exception(f"Unsupported lifestyle: {lifestyle}")

    income_left_for_mortgage_per_month = income_left_for_mortgage / 12
    return income_left_for_mortgage_per_month

In [ ]:
def compute_highest_interest_rate_affordable(
    salary,
    pension_contributions_rate,
    annual_council_tax,
    annual_home_insurance,
    annual_ground_rent,
    annual_service_charge,
    total_mortgage_size,
    total_mortgage_length_in_years,
    lifestyle,
    annual_additional_minimum_living_costs,
    annual_bougie_lifestyle_costs,
    annual_inflated_additional_minimum_living_costs
):
    income_left_for_mortgage_per_month = compute_monthly_income_left_for_mortgage_per_month(
        salary,
        pension_contributions_rate,
        annual_council_tax,
        annual_home_insurance,
        annual_ground_rent,
        annual_service_charge,
        lifestyle,
        annual_additional_minimum_living_costs,
        annual_bougie_lifestyle_costs,
        annual_inflated_additional_minimum_living_costs
    )
    
    starting_interest_rate = 0.0025
    interest_rate = starting_interest_rate
    interest_rate_incrementation = 0.0025

    # LOL
    while True:
        monthly_mortgage_repayments = compute_monthly_mortgage_repayments(
            total_mortgage_size, total_mortgage_length_in_years, interest_rate
        )

        if monthly_mortgage_repayments > income_left_for_mortgage_per_month:
            if interest_rate == starting_interest_rate:
                return "N/A"
            return 100*(interest_rate - interest_rate_incrementation)

        interest_rate = interest_rate + interest_rate_incrementation
    

In [ ]:
def compute_pivot_table(
    df,
    pivot_row,
    pivot_columns,
    annual_council_tax,
    annual_home_insrance,
    annual_ground_rent,
    annual_service_charge,
    mortgage_length_in_years,
    print_assumptions=True
):
    if print_assumptions:
        print('Assumptions: (only my payments)')
        print(f'- Mortgage length in years: {mortgage_length_in_years}')
        print(f'- Annual council tax: {annual_council_tax}')
        print(f'- Annual home insurance: {annual_home_insrance}')
        print(f'- Annual ground rent: {annual_ground_rent}')
        print(f'- Annual service charge: {annual_service_charge}')

    return df.pivot(index=pivot_row, columns=pivot_columns)

In [ ]:

salaries = [salary for salary in range(60_000, 130_001, 5_000)]
mortgage_sizes = [mortgage_size for mortgage_size in range(270_000, 280_001, 10_000)]
lifestyles = ["inflated_75-25", "50-30-20", "35p_net_on_mortgage", "25p_net_on_mortgage", "inflated_essentials", "bougie"]

combinations = list(product(salaries, mortgage_sizes, lifestyles))

df = pd.DataFrame(combinations, columns=['salary', 'mortgage_size', 'lifestyle'])

df["max_interest_rate_affordable"] = df.apply(lambda x: compute_highest_interest_rate_affordable(
    x["salary"],
    annual_pension_contributions_rate,
    annual_council_tax,
    annual_home_insrance,
    annual_ground_rent,
    annual_service_charge,
    x["mortgage_size"],
    mortgage_length_in_years,
    x["lifestyle"],
    annual_minimum_additional_living_costs,
    annual_inflated_additional_minimum_additional_living_costs
), axis=1)

df = display_matrix(
    df,
    "salary",
    ["mortgage_size", "lifestyle"],
    annual_council_tax,
    annual_home_insrance,
    annual_ground_rent,
    annual_service_charge,
    mortgage_length_in_years
)

df = df.astype(float).round(2)
df.to_excel("data/mortgage_affordability_by_salary.xlsx", freeze_panes=(1, 1))

df